## 1 :- Fake News
##   0 :- Real News

Imporing the Dependencies

In [34]:
import pandas as pd
import numpy as np
import re # re=> regular expression for searching the text from para 
from nltk.corpus import stopwords  #nltk=> Natural language tool kit , corpus => body of the text 
from nltk.stem.porter import PorterStemmer #stem => to remove prefix and sufix from word
from sklearn.feature_extraction.text import TfidfVectorizer #To convert text into number or feature data
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [35]:
# downloading stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cognitq/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

stopwords in english

In [36]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Data Pre-processing

In [37]:
# loading the dataset to pandas dataframe
news_dataset = pd.read_csv('./train.csv')

In [38]:
news_dataset.shape

(20800, 5)

In [39]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [40]:
# counting the number of missing values in datasets in each column
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

title have 558 null values like so on 

In [41]:
# replacing the null values with the empty_string 
news_dataset = news_dataset.fillna('')

In [42]:
# merging the auther name and title (we can also add text but it will take more time)
news_dataset['content']= news_dataset['author'] + ' ' + news_dataset['title']

In [43]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


## seperating the data and label 

In [44]:
# no need to seperat data on this stage
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [45]:
print(X)
print(Y)


          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

## stemming:-
    process of reducing a word into its root word
    ex:- acting, actor, actress => act will be remove

In [46]:
port_stem = PorterStemmer()

In [47]:
def steming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content) #remove all the things which are not from a-z and A-Z ex:- numbers and special symbols will be remove
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split() # words will be convert into lists 
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english') ] #steming + remoing stopwords
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content


In [48]:
news_dataset['content'] = news_dataset['content'].apply(steming)

In [49]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


## seperating data base on content column

In [50]:
X = news_dataset['content'].values
Y = news_dataset['label'].values


In [51]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [52]:
print(Y)

[1 0 1 ... 0 1 1]


In [53]:
X.shape

(20800,)

In [54]:
Y.shape

(20800,)

# converting the text data into numerical data

In [55]:
vectorizer = TfidfVectorizer() # Tf => Term-frequency, idf=> inverse document frequency 
# counts the number of times the word appers  

In [56]:
vectorizer.fit(X)
X = vectorizer.transform(X)

In [57]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

# Splitting the dataset into training and testing

Traing data where 
    test_size = 0.2(20% of all data)
    stratify = Y (base on 0 1 label)
    random_state= 42(To reproduce the data randomly in test and train i Type 42 so data will reproduce after 42 times of running if i type 10 then data will reproduce after 10 time)

In [58]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42)

# model training

In [59]:
model = LogisticRegression()


In [60]:
model.fit(X_train, Y_train)

LogisticRegression()

# Evaluation

Accuracy Score

In [61]:
# Accuracy on training data
# predication of data
X_train_predict = model.predict(X_train)#ex:- X_train_prdication=> our ansewr on solveing question paper 
# Training data  Accuracy
training_data_accuracy = accuracy_score(X_train_predict, Y_train)#ex:- Y_train is Correct ans. finding our score 

In [62]:
print("Accuracy on Training data : - ",training_data_accuracy)

Accuracy on Training data : -  0.9874399038461539


we got 98.74% of Accuracy


In [63]:
# Accuracy on test data
# predication of data
X_test_prdication = model.predict(X_test) #ex:- X_test_prdication=> our ansewr on solveing question paper 
# Testing data  Accuracy
testing_data_accuracy = accuracy_score(X_test_prdication, Y_test) #ex:- Y_train is Correct ans. finding our score 

In [64]:
print("Accuracy on Testing data : - ",testing_data_accuracy)


Accuracy on Testing data : -  0.9752403846153846


# Making A prediction sysytem

In [74]:
X_new = X_test[2]

prediction = model.predict(X_new)
print(prediction)

[1]


In [75]:
# printing data or output in human readable format
if(prediction[0]==0):
    print("The news is Real")
else:
    print("The news is fake")

The news is fake


In [76]:
print(Y_test[2])


1
